Create retriever

In [1]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

In [2]:
document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

WARNING - elasticsearch -  HEAD http://localhost:9200/ [status:N/A request:0.004s]
Traceback (most recent call last):
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/http_urllib3.py", line 245, in perform_request
    response = self.pool.urlopen(
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 785, in urlopen
    retries = retries.increment

WARNING - elasticsearch -  HEAD http://localhost:9200/ [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/http_urllib3.py", line 245, in perform_request
    response = self.pool.urlopen(
  File "/home/serapf/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 785, in urlopen
    retries = retries.increment

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

Make the google query

In [82]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from langdetect import detect


query = "history of japan"

valid_url = []

searches = ["what is hatsune miku", "hatsune miku", "hatsune miku"]

# to search
    
headers = {'Accept-Encoding': 'identity'}

for query in searches:
    r = requests.get("https://www.google.com/search?q="+query+"&start=0", headers=headers)

    tree = html.fromstring(r.content)
    urls = tree.xpath('//div[@class="egMi0 kCrYT"]/a[@href]/@href')
    urls = [url[7:].split("&")[0] for url in urls]
    
    for url in urls:
        
        if url not in valid_url:
            valid_url.append(url)
valid_url

['https://en.wikipedia.org/wiki/Hatsune_Miku',
 'https://es.wikipedia.org/wiki/Miku_Hatsune',
 'https://www.crypton.co.jp/miku_eng',
 'https://vocaloid.fandom.com/wiki/Hatsune_Miku',
 'https://vocaloid.fandom.com/es/wiki/Hatsune_Miku',
 'https://www.nippon.com/es/japan-topics/g01038/',
 'https://www.definitions.net/definition/hatsune%2Bmiku',
 'https://www.dictionary.com/e/fictional-characters/hatsune-miku/',
 'https://www.youtube.com/watch%3Fv%3DberD3I2RXTM',
 'https://www.wmagazine.com/story/hatsune-miku-crypton-future',
 'https://www.youtube.com/watch%3Fv%3D66J_A2__tl0',
 'https://www.amazon.com.mx/Paquete-Vocaloid4-Hatsune-Miku-V4X/dp/B01G6XNQ5A',
 'https://miku.sega.com/megamixplus/%3Flang%3Des',
 'https://www.nintendo.com/es-mx/store/products/hatsune-miku-project-diva-mega-mix-switch/']

In [78]:


pages = []

min_words = 500
    
for url in valid_url:

    r = requests.get(url)
    print(r)
    
    innerHtml = r.content

    ##Use BeautifulSoup to convert the html file to plain text
    soup = BeautifulSoup(innerHtml, "html.parser")
    html_text = soup.get_text()
    html_text = re.sub('(?<=\n)\s+\n', '\n', html_text)
    html_text = re.sub('\s{2,}', '\n', html_text)
    html_text = re.sub('\[\d+]', '', html_text)


    #Detect if the text is in english
    language = detect(html_text)

    if len(html_text.split(" ")) > min_words and language == 'en':
          ##Add the plain text to the rawdata list
          pages.append(html_text)


<Response [200]>
<Response [429]>
<Response [200]>


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<Response [200]>
<Response [200]>


In [12]:
import requests
r = requests.get("https://google.com")
r.ok

True

In [4]:
help(search)

Help on function search in module googlesearch:

search(query, tld='com', lang='en', tbs='0', safe='off', num=10, start=0, stop=None, pause=2.0, country='', extra_params=None, user_agent=None, verify_ssl=True)
    Search the given query string using Google.
    
    :param str query: Query string. Must NOT be url-encoded.
    :param str tld: Top level domain.
    :param str lang: Language.
    :param str tbs: Time limits (i.e "qdr:h" => last hour,
        "qdr:d" => last 24 hours, "qdr:m" => last month).
    :param str safe: Safe search.
    :param int num: Number of results per page.
    :param int start: First result to retrieve.
    :param int stop: Last result to retrieve.
        Use None to keep searching forever.
    :param float pause: Lapse to wait between HTTP requests.
        A lapse too long will make the search slow, but a lapse too short may
        cause Google to block your IP. Your mileage may vary!
    :param str country: Country or region to focus the search on. Sim

In [10]:
import requests

requests.get("http://www.google.com?")

<Response [200]>

In [231]:
valid_url

['https://en.wikipedia.org/wiki/Trigonometry',
 'https://www.britannica.com/science/trigonometry',
 'https://www.mathsisfun.com/algebra/trigonometry.html',
 'https://www.youtube.com/watch?v=PUB0TaZ7bhA',
 'https://www.cuemath.com/trigonometry/',
 'https://byjus.com/maths/trigonometry/',
 'https://www.skillsyouneed.com/num/trigonometry.html',
 'https://www2.clarku.edu/faculty/djoyce/trig/',
 'https://mathworld.wolfram.com/Trigonometry.html',
 'https://www.geogebra.org/t/trigonometry?lang=en',
 'https://www.govst.edu/uploadedFiles/Academics/Colleges_and_Programs/CAS/Trigonometry_Short_Course_Tutorial_Lauren_Johnson.pdf',
 'https://www.livescience.com/51026-what-is-trigonometry.html',
 'https://www.wolframalpha.com/examples/mathematics/trigonometry',
 'https://www.sfu.ca/math-coursenotes/Math%20157%20Course%20Notes/sec_Trigonometry.html',
 'https://www.omnicalculator.com/math/trigonometry']

In [232]:
import urllib
from bs4 import BeautifulSoup
from langdetect import detect
import requests

import re

min_words = 400

pages = []

for url in valid_url:
    
    innerHtml = requests.get(url).content
    
    ##Use BeautifulSoup to convert the html file to plain text
    soup = BeautifulSoup(innerHtml, "html.parser")
    html_text = soup.get_text()
    html_text = re.sub('(?<=\n)\s+\n', '\n', html_text)
    html_text = re.sub('\s{2,}', '\n', html_text)
    html_text = re.sub('\[.]', '', html_text)

    
    #Detect if the text is in english
    language = detect(html_text)
    
    if len(html_text.split(" ")) > min_words and language == 'en':
          ##Add the plain text to the rawdata list
          pages.append(html_text)



WARNING - bs4.dammit -  Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [233]:
pages[2]

'\nTrigonometry\nIntroduction to Trigonometry\nTrigonometry (from Greek trigonon "triangle" + metron "measure")\nWant to learn Trigonometry? Here is a quick summary.\nFollow the links for more, or go to Trigonometry Index\nTrigonometry ... is all about triangles.\nTrigonometry helps us find angles and distances, and is used a lot in science, engineering, video games, and more!\nRight-Angled Triangle\nThe triangle of most interest is the right-angled triangle. The right angle is shown by the little box in the corner:\nAnother angle is often labeled θ, and the three sides are then called:\nAdjacent: adjacent (next to) the angle θ\nOpposite: opposite the angle θ\nand the longest side is the Hypotenuse\nWhy a Right-Angled Triangle?\nWhy is this triangle so important?\nImagine we can measure along and up but want to know the direct distance and angle:\nTrigonometry can find that missing angle and distance.\nOr maybe we have a distance and angle and need to "plot the dot" along and up:\nQues

Paragraph selector

In [234]:
min_words_per_paragraph = 40

contexts = []

for page in pages:
    
    paragraphs = page.split("\n")
    
    for paragraph in paragraphs:
        
        size = len(paragraph.split(" "))
    
        if size >= min_words_per_paragraph:
            contexts.append(paragraph)
    


In [235]:
len(contexts)

96

In [236]:
contexts[50]

'In Diagram ii, we have rotated the radius further in an anti-clockwise direction, past the vertical (y axis) into the next quadrant. Here θ is an obtuse angle, between 90° and 180°. The reference angle alpha α is equal to 180° − θ, and is the acute angle within the right-angled triangle.'

In [237]:
squad_docs = []

for context in contexts:
    squad_docs.append({
        'content': context
    })

Put data in

In [238]:
document_store.write_documents(squad_docs)

Retrive

In [239]:
from haystack.retriever.sparse import BM25Retriever

retriever = BM25Retriever(document_store)

In [69]:
retriever.retrieve("When was UNAM created?")

NameError: name 'retriever' is not defined

In [211]:


res = requests.post('http://localhost:9200/squad_docs/_delete_by_query',
                    json={
                        'query': {
                            'match_all': {}
                        }
                    })

res.json()

{'took': 1,
 'timed_out': False,
 'total': 0,
 'deleted': 0,
 'batches': 0,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [67]:


import spacy

text = "What is sadness?"

text = text.lower()

# First search - complete question

print(text)            
# Second search - search phrase
                   
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

phrase = ' '.join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

print(phrase)

# Third search - Nouns in phrase



what is sadness?
sadness


In [68]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

list_of_names = []
current = False

for token in doc:
    
    if token.pos_ == 'PROPN' or token.pos_ == "NOUN" or token.pos_ == "ADJ":
        
        if not current:
            current = True
            list_of_names.append(token.text)
        else:
            list_of_names[-1] = list_of_names[-1] + " " + token.text
    else:
        current = False
    

print(list_of_names)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    

['sadness']
what what PRON WP nsubj xxxx True True
is be AUX VBZ ROOT xx True True
sadness sadness NOUN NN acomp xxxx True False
? ? PUNCT . punct ? False False


In [51]:
len(doc)

8